In [2]:
%matplotlib inline


Link Prediction using Graph Neural Networks
===========================================

In the :doc:`introduction <1_introduction>`, you have already learned
the basic workflow of using GNNs for node classification,
i.e. predicting the category of a node in a graph. This tutorial will
teach you how to train a GNN for link prediction, i.e. predicting the
existence of an edge between two arbitrary nodes in a graph.

By the end of this tutorial you will be able to

-  Build a GNN-based link prediction model.
-  Train and evaluate the model on a small DGL-provided dataset.

(Time estimate: 28 minutes)


In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

Using backend: pytorch


In [357]:
with open('citeseer.content', 'r') as f:
    feat_new = f.readlines()

In [363]:
with open('citeseer.cites', 'r') as f:
    cites_new = f.readlines()

In [360]:
features_new = {item.split('\t')[0]: _ for _, item in enumerate(feat_new)}

In [377]:
li_neww = []
for item in unlabeled_edges_list:
    it = item['source'], item['target']
    if it in cites_new_ or (it[1], it[0]) in cites_new_:
        li_neww.append(1)
    else:
        li_neww.append(0)

In [374]:
cites_new_ = [(features_new.get(item.strip().split('\t')[0], -1), features_new.get(item.strip().split('\t')[1], -1)) for item in cites_new]

Overview of Link Prediction with GNN
------------------------------------

Many applications such as social recommendation, item recommendation,
knowledge graph completion, etc., can be formulated as link prediction,
which predicts whether an edge exists between two particular nodes. This
tutorial shows an example of predicting whether a citation relationship,
either citing or being cited, between two papers exists in a citation
network.

This tutorial follows a relatively simple practice from
`SEAL <https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf>`__.
It formulates the link prediction problem as a binary classification
problem as follows:

-  Treat the edges in the graph as *positive examples*.
-  Sample a number of non-existent edges (i.e. node pairs with no edges
   between them) as *negative* examples.
-  Divide the positive examples and negative examples into a training
   set and a test set.
-  Evaluate the model with any binary classification metric such as Area
   Under Curve (AUC).

In some domains such as large-scale recommender systems or information
retrieval, you may favor metrics that emphasize good performance of
top-K predictions. In these cases you may want to consider other metrics
such as mean average precision, and use other negative sampling methods,
which are beyond the scope of this tutorial.

Loading graph and features
--------------------------

Following the :doc:`introduction <1_introduction>`, this tutorial
first loads the Cora dataset.




In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


Prepare training and testing sets
---------------------------------

This tutorial randomly picks 10% of the edges for positive examples in
the test set, and leave the rest for the training set. It then samples
the same number of edges for negative examples in both sets.




In [341]:
import networkx as nx
import numpy as np
import dgl
with open('labeled_edges.txt', 'r') as f:
    labeled_edges = f.readlines()
with open('unlabeled_edges.txt', 'r') as f:
    unlabeled_edges = f.readlines()
with open('features.txt', 'r') as f:
    features = f.readlines()
# features_full = [{'node': int(line[0]), 'features': ' '.join(line[1:])} for line in [line.strip().split() for line in features]]
unlabeled_edges_list = [{'source': int(line[0]), 'target': int(line[1])} for line in [line.strip().split() for line in unlabeled_edges]]
labeled_edges_list = [{'source': int(line[0]), 'target': int(line[1]), 'label': int(line[2])} for line in [line.strip().split() for line in labeled_edges]]
# nodes = [item['node'] for item in features_full]
features = [list(map(int, item.split())) for item in features]
features = np.array([list(map(float, item)) for item in features])
# G = nx.Graph()
# G.add_nodes_from(nodes)
# G.add_weighted_edges_from([(item['source'], item['target'], item['label']) for item in labeled_edges_list])
# G.add_weighted_edges_from([(item['target'], item['source'], item['label']) for item in labeled_edges_list])

# src = np.random.randint(0, 100, 500)
# dst = np.random.randint(0, 100, 500)
src = np.array([item['source'] for item in labeled_edges_list if item['label'] == 1])
dst = np.array([item['target'] for item in labeled_edges_list if item['label'] == 1])
src2 = np.array([item['source'] for item in labeled_edges_list if item['label'] == 0])
dst2 = np.array([item['target'] for item in labeled_edges_list if item['label'] == 0])
label = np.array([item['label'] for item in labeled_edges_list])
# src2 = np.array([item['source'] for item in unlabeled_edges_list])
# dst2 = np.array([item['target'] for item in unlabeled_edges_list])
# make it symmetric

edge_pred_graph = dgl.graph((np.concatenate([src, dst]), np.concatenate([dst, src])))
# # synthetic node and edge features, as well as edge labels
edge_pred_graph.ndata['feat'] = torch.FloatTensor(features)
# edge_pred_graph.edata['feature'] = torch.randn(1000, 10)
# edge_pred_graph.edata['label'] = torch.tensor(np.concatenate([label, label]))
# # synthetic train-validation-test splits
# edge_pred_graph.edata['train_mask'] = torch.zeros(2 * label.shape[0], dtype=torch.bool).bernoulli(0.6)

g = edge_pred_graph

In [48]:
# with open('features.txt', 'r') as f:
#     features = f.readlines()
# features_full = [{'node': int(line[0]), 'features': ' '.join(line[1:])} for line in [line.strip().split() for line in features]]
# nodes = [item['node'] for item in features_full]
# G = nx.Graph()
# G.add_nodes_from(nodes)
# G.add_weighted_edges_from([(item['source'], item['target'], item['label']) for item in labeled_edges_list])
# G.add_weighted_edges_from([(item['target'], item['source'], item['label']) for item in labeled_edges_list])

In [70]:
# # Split edge set for training and testing
# u1 = []
# v1 = []
# for edge in G.edges():
#     u1.append(int(edge[0]))
#     v1.append(int(edge[1]))
#     u1.append(int(edge[1]))
#     v1.append(int(edge[0]))
# # for edge in G.edges():
# u1 = torch.tensor(u1, dtype=torch.int32)
# v1 = torch.tensor(v1, dtype=torch.int32)
    

# eids = np.arange(2 * len(G.edges()))
# eids = np.random.permutation(eids)
# test_size = int(len(eids) * 0.1)
# train_size = 2 * len(G.edges()) - test_size
# test_pos_u, test_pos_v = u1[eids[:test_size]], v1[eids[:test_size]]
# train_pos_u, train_pos_v = u1[eids[test_size:]], v1[eids[test_size:]]

# # Find all negative edges and split them for training and testing
# adj = sp.coo_matrix((np.ones(len(u1)), (u1.numpy(), v1.numpy())))
# adj_neg = 1 - adj.todense() - np.eye(len(G.nodes()))
# neg_u, neg_v = np.where(adj_neg != 0)

# neg_eids = np.random.choice(len(neg_u), 2 * len(G.edges()) // 2)
# test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
# train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [71]:
# src = np.array([item['source'] for item in labeled_edges_list])
# dst = np.array([item['target'] for item in labeled_edges_list])
# label = np.array([item['label'] for item in labeled_edges_list])
# # src2 = np.array([item['source'] for item in unlabeled_edges_list])
# # dst2 = np.array([item['target'] for item in unlabeled_edges_list])
# # make it symmetric

# edge_pred_graph = dgl.graph((np.concatenate([src, dst]), np.concatenate([dst, src])))
# # # synthetic node and edge features, as well as edge labels
# edge_pred_graph.ndata['feature'] = torch.tensor(features)
# # edge_pred_graph.edata['feature'] = torch.randn(1000, 10)
# edge_pred_graph.edata['label'] = torch.tensor(np.concatenate([label, label]))
# # # synthetic train-validation-test splits
# edge_pred_graph.edata['train_mask'] = torch.zeros(2 * label.shape[0], dtype=torch.bool).bernoulli(0.6)

In [4]:
# Нужно запихнуть все положительные в pos а отрицательные в neg

In [3]:
# neg_u.shape

In [343]:
# Split edge set for training and testing
u_, v_ = g.edges()
u = u_#[:2*len(src)]
v = v_#[:2*len(src)]

eids = np.arange(2 * len(src))
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.2)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]
# train_pos_u, train_pos_v = u, v

# Find all negative edges and split them for training and testing
# adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
# adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
# neg_u, neg_v = np.where(adj_neg != 0)
# neg_u = u_[2*len(src):]
# neg_v = v_[2*len(src):]
neg_u = src2
neg_v = dst2

# neg_eids = np.random.choice(len(neg_u), g.number_of_edges() // 2)
test_neg_u, test_neg_v = neg_u[:test_size // 2], neg_v[: test_size // 2 ]
train_neg_u, train_neg_v = neg_u[test_size // 2:], neg_v[test_size // 2:]
# test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
# train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

# train_neg_u, train_neg_v = neg_u, neg_v

In [356]:
len(features)

3312

When training, you will need to remove the edges in the test set from
the original graph. You can do this via ``dgl.remove_edges``.

<div class="alert alert-info"><h4>Note</h4><p>``dgl.remove_edges`` works by creating a subgraph from the
   original graph, resulting in a copy and therefore could be slow for
   large graphs. If so, you could save the training and test graph to
   disk, as you would do for preprocessing.</p></div>




In [62]:
# for item in zip(train_pos_u, train_pos_v):
#     print(item)

In [63]:
# (7, 616) in G.edges

In [344]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [68]:
# train_g, train_g.ndata['feat']

Define a GraphSAGE model
------------------------

This tutorial builds a model consisting of two
`GraphSAGE <https://arxiv.org/abs/1706.02216>`__ layers, each computes
new node representations by averaging neighbor information. DGL provides
``dgl.nn.SAGEConv`` that conveniently creates a GraphSAGE layer.




In [345]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, dropout):
        super(GraphSAGE, self).__init__()
        self.r = nn.LeakyReLU(0.1)
        self.conv1 = SAGEConv(in_feats, h_feats*2, 'gcn')
        self.conv2 = SAGEConv(h_feats*2, h_feats, 'gcn')
        self.conv3 = SAGEConv(h_feats, h_feats, 'gcn')
#         self.conv4 = SAGEConv(h_feats, h_feats, 'gcn')
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0.
    
    def forward(self, g, in_feat, mode='eval'):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        if mode == 'train':
            h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
#         h = F.relu(h)
#         h = self.conv4(g, h)
        return h

The model then predicts the probability of existence of an edge by
computing a score between the representations of both incident nodes
with a function (e.g. an MLP or a dot product), which you will see in
the next section.

\begin{align}\hat{y}_{u\sim v} = f(h_u, h_v)\end{align}




Positive graph, negative graph, and ``apply_edges``
---------------------------------------------------

In previous tutorials you have learned how to compute node
representations with a GNN. However, link prediction requires you to
compute representation of *pairs of nodes*.

DGL recommends you to treat the pairs of nodes as another graph, since
you can describe a pair of nodes with an edge. In link prediction, you
will have a *positive graph* consisting of all the positive examples as
edges, and a *negative graph* consisting of all the negative examples.
The *positive graph* and the *negative graph* will contain the same set
of nodes as the original graph.  This makes it easier to pass node
features among multiple graphs for computation.  As you will see later,
you can directly fed the node representations computed on the entire
graph to the positive and the negative graphs for computing pair-wise
scores.

The following code constructs the positive graph and the negative graph
for the training set and the test set respectively.




In [346]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

The benefit of treating the pairs of nodes as a graph is that you can
use the ``DGLGraph.apply_edges`` method, which conveniently computes new
edge features based on the incident nodes’ features and the original
edge features (if applicable).

DGL provides a set of optimized builtin functions to compute new
edge features based on the original node/edge features. For example,
``dgl.function.u_dot_v`` computes a dot product of the incident nodes’
representations for each edge.




In [347]:
src_new = torch.tensor([item['source'] for item in unlabeled_edges_list])
dst_new = torch.tensor([item['target'] for item in unlabeled_edges_list])
new_inf = dgl.graph((src_new, dst_new), num_nodes=g.number_of_nodes())

In [536]:
# import dgl.function as fn

# class DotPredictor(nn.Module):
#     def forward(self, g, h):
#         with g.local_scope():
#             g.ndata['h'] = h
#             # Compute a new edge feature named 'score' by a dot-product between the
#             # source node feature 'h' and destination node feature 'h'.
#             g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
#             # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
#             return g.edata['score'][:, 0]

You can also write your own function if it is complex.
For instance, the following module produces a scalar score on each edge
by concatenating the incident nodes’ features and passing it to an MLP.




In [348]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        score = torch.sigmoid(self.W2(F.relu(self.W1(h))).squeeze(1))
#         score = self.W2(F.relu(self.W1(h))).squeeze(1)
        return {'score': score}

    def forward(self, g, h, d=False):
        with g.local_scope():
#             if d:
#                 h = nn.functional.dropout(h, p=0.1)
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [349]:
class FullModel(nn.Module):
    def __init__(self, feat_in, h_feats, dropout):
        super().__init__()
        self.sage = GraphSAGE(feat_in, h_feats, dropout)
        self.pred = MLPPredictor(h_feats)
    
    def forward(self, train_g, features, pos, neg, mode="eval"):
        h = self.sage(train_g, features, mode)
        pos_score = self.pred(pos, h, True)
        neg_score = self.pred(neg, h, True)
        return pos_score, neg_score

In [353]:

# in this case, loss will in training loop
model = FullModel(train_g.ndata['feat'].shape[1], 16, 0.01)
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
best_val_loss = 1
for e in range(160):
    # forward
    pos_score, neg_score = model(train_g, train_g.ndata['feat'], train_pos_g, train_neg_g, mode="train")
    loss = compute_loss(pos_score, neg_score)

    with torch.no_grad():
        pos_score, neg_score = model(train_g, train_g.ndata['feat'], test_pos_g, test_neg_g)
        val_loss = compute_loss(pos_score, neg_score)
        if val_loss < best_val_loss:
            best_model = model
            best_val_loss = val_loss
#     if val_loss < 0.45 or compute_auc(pos_score, neg_score) > 0.75:
#         print('In epoch {}, loss: {}, val_loss: {}, AUC: {}'.format(e, loss, val_loss, compute_auc(pos_score, neg_score)))
#         best_model = model
#         break

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#     if e % 1 == 4:
#         print('In epoch {}, loss: {}'.format(e, loss))
    print('In epoch {}, loss: {}, val_loss: {}, AUC: {}'.format(e, loss, val_loss, compute_auc(pos_score, neg_score)))

# with torch.no_grad():
#     pos_score = pred(test_pos_g, h)
#     neg_score = pred(test_neg_g, h)
#     print('AUC', compute_auc(pos_score, neg_score), compute_loss(pos_score, neg_score))

In epoch 0, loss: 5.697616100311279, val_loss: 5.299469947814941, AUC: 0.49667774086378735
In epoch 1, loss: 23.22168731689453, val_loss: 22.32206153869629, AUC: 0.523671096345515
In epoch 2, loss: 22.80771827697754, val_loss: 22.5015926361084, AUC: 0.5
In epoch 3, loss: 11.896060943603516, val_loss: 10.815698623657227, AUC: 0.5
In epoch 4, loss: 2.8950796127319336, val_loss: 2.74806547164917, AUC: 0.5
In epoch 5, loss: 0.7994735836982727, val_loss: 0.7475866675376892, AUC: 0.5240863787375416
In epoch 6, loss: 0.8651487827301025, val_loss: 0.8556873798370361, AUC: 0.48588039867109634
In epoch 7, loss: 0.6579791903495789, val_loss: 0.6764896512031555, AUC: 0.5
In epoch 8, loss: 0.7037294507026672, val_loss: 0.684185266494751, AUC: 0.5182724252491694
In epoch 9, loss: 0.634511411190033, val_loss: 0.6363199353218079, AUC: 0.5477574750830565
In epoch 10, loss: 0.6806454658508301, val_loss: 0.681261420249939, AUC: 0.5577242524916943
In epoch 11, loss: 0.6348400115966797, val_loss: 0.6341421

In epoch 89, loss: 0.37366679310798645, val_loss: 0.4904834032058716, AUC: 0.7109634551495017
In epoch 90, loss: 0.3739759027957916, val_loss: 0.48571476340293884, AUC: 0.7101328903654485
In epoch 91, loss: 0.3673158586025238, val_loss: 0.48788735270500183, AUC: 0.7138704318936877
In epoch 92, loss: 0.370735764503479, val_loss: 0.4971090853214264, AUC: 0.7122093023255814
In epoch 93, loss: 0.3752100169658661, val_loss: 0.5060659050941467, AUC: 0.7138704318936877
In epoch 94, loss: 0.3593093454837799, val_loss: 0.49367672204971313, AUC: 0.715531561461794
In epoch 95, loss: 0.3559504449367523, val_loss: 0.49130144715309143, AUC: 0.7180232558139534
In epoch 96, loss: 0.3471277952194214, val_loss: 0.4977213442325592, AUC: 0.723421926910299
In epoch 97, loss: 0.35855183005332947, val_loss: 0.5104701519012451, AUC: 0.7221760797342193
In epoch 98, loss: 0.3627108037471771, val_loss: 0.5058494210243225, AUC: 0.7279900332225914
In epoch 99, loss: 0.3632584512233734, val_loss: 0.5002349615097046

<div class="alert alert-info"><h4>Note</h4><p>The builtin functions are optimized for both speed and memory.
   We recommend using builtin functions whenever possible.</p></div>

<div class="alert alert-info"><h4>Note</h4><p>If you have read the :doc:`message passing
   tutorial <3_message_passing>`, you will notice that the
   argument ``apply_edges`` takes has exactly the same form as a message
   function in ``update_all``.</p></div>




Training loop
-------------

After you defined the node representation computation and the edge score
computation, you can go ahead and define the overall model, loss
function, and evaluation metric.

The loss function is simply binary cross entropy loss.

\begin{align}\mathcal{L} = -\sum_{u\sim v\in \mathcal{D}}\left( y_{u\sim v}\log(\hat{y}_{u\sim v}) + (1-y_{u\sim v})\log(1-\hat{y}_{u\sim v})) \right)\end{align}

The evaluation metric in this tutorial is AUC.




In [264]:
from sklearn.metrics import roc_auc_score
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16, 0.3)
# You can replace DotPredictor with MLPPredictor.
pred = MLPPredictor(16)
# pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
#     print(scores)
#     scores = torch.sigmoid(scores)
#     print(scores)
    return F.binary_cross_entropy(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score > 0.5, neg_score > 0.5]).numpy()
#     print(type(scores))
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
#     scores = F.softmax(scores)
    return roc_auc_score(labels, scores)

The training loop goes as follows:

<div class="alert alert-info"><h4>Note</h4><p>This tutorial does not include evaluation on a validation
   set. In practice you should save and evaluate the best model based on
   performance on the validation set.</p></div>




In [292]:
# ----------- 3. set up loss and optimizer -------------- #
for _ in range(100):
    # in this case, loss will in training loop
    model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
    # You can replace DotPredictor with MLPPredictor.
    pred = MLPPredictor(16)
    optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001)

    # ----------- 4. training -------------------------------- #
    all_logits = []
    for e in range(80):
        # forward
        h = model(train_g, train_g.ndata['feat'], mode="train")
        pos_score = pred(train_pos_g, h, True)
        neg_score = pred(train_neg_g, h, True)
        loss = compute_loss(pos_score, neg_score)

        with torch.no_grad():
            pos_score = pred(test_pos_g, h)
            neg_score = pred(test_neg_g, h)
            val_loss = compute_loss(pos_score, neg_score)
#         if val_loss < 0.45 or compute_auc(pos_score, neg_score) > 0.75:
#             print('In epoch {}, loss: {}, val_loss: {}, AUC: {}'.format(e, loss, val_loss, compute_auc(pos_score, neg_score)))
#             best_model = model
#             break

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
    #         print('In epoch {}, loss: {}'.format(e, loss))
            print('In epoch {}, loss: {}, val_loss: {}, AUC: {}'.format(e, loss, val_loss, compute_auc(pos_score, neg_score)))


# ----------- 5. check results ------------------------ #
# def compute_loss(pos_score, neg_score):
#     scores = torch.cat([pos_score, neg_score])
#     labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
# #     scores = F.sigmoid(scores)
#     return F.binary_cross_entropy(scores, labels)

# def compute_auc(pos_score, neg_score):
#     scores = torch.cat([pos_score > 0.5, neg_score > 0.5]).numpy()
# #     print(type(scores))
#     labels = torch.cat(
#         [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
# #     scores = F.softmax(scores)
#     return roc_auc_score(labels, scores)
# from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score), compute_loss(pos_score, neg_score))

TypeError: __init__() missing 1 required positional argument: 'dropout'

In [523]:
h[0]

tensor([ 20.0644, -22.6706,  11.8541,   5.1292,   3.2398,  -6.1742, -25.9686,
         11.6753, -29.2209,   7.1215], grad_fn=<SelectBackward>)

In [553]:
test_pos_g

Graph(num_nodes=3312, num_edges=1505,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
# 3 gcn 16 100 subm3_test 0.7

In [354]:
with torch.no_grad():
    new_pred, _ = best_model(train_g, train_g.ndata['feat'], new_inf, new_inf)

In [178]:
new_pred = pred(new_inf, h)

In [379]:
# res1 = [1 if item > 0.5 else 0 for item in new_pred] 
with open('ideal.txt', 'w') as f:
    for item in li_neww:
        f.write(str(item) + '\n')

In [137]:
sorted(pos_score)[607]

tensor(2.0385)

In [139]:
sorted(neg_score)[607]

tensor(-1.7167)

In [126]:
pos_score.mean(), neg_score.mean()

(tensor(0.8315), tensor(0.4181))